In [ ]:
from py2neo import Graph

neo_host = "neo4j"
neo_port = 7687

graph = Graph(f"bolt://{neo_host}:{neo_port}")

In [ ]:
NB_SELECTED = 4

graph.run("""MATCH ()-[r:Dissed]->(target)
                RETURN target, count(r) AS num
                ORDER BY num DESC
                LIMIT $nb_selected
            """, parameters={"nb_selected": NB_SELECTED}).to_data_frame()[['target']]

In [ ]:
NB_SELECTED = 4

graph.run("""MATCH (artist)-[r:Dissed]->()
                RETURN artist, count(r) AS num
                ORDER BY num DESC
                LIMIT $nb_selected
            """, parameters={"nb_selected": NB_SELECTED}).to_data_frame()[['artist']]